# The national lottery of Israel (Mifal HaPayis) Prediction using LSTM

The purpose of this notebook is to predict the Israeli lottorey results by the LSTM model. (For Mifal HaPayis Results: https://www.pais.co.il/Lotto/lotto_resultsDownload.aspx)

## The libraries we will work with

In [ ]:
import pandas as pd; print (pd.__version__)
import numpy as np; print (np.__version__)
import os
from pathlib import Path
from sklearn.preprocessing import StandardScaler

## Prepare/Generate data set

First, we load into our system the latest results of the lottery games in the input folder

In [ ]:
filename = 'israeli-lotto' 

df = pd.read_csv('../input/israelilotto/'+filename+'.csv')
df.columns = ['id','date','A', 'B', 'C', 'D', 'E', 'F', 'Bonus', 'NumWin', 'NumWinEx', 'empty'] #rename from hebrew to english
df = df.drop(['id','date', 'NumWin', 'NumWinEx', 'empty'], axis = 1) #remove columns
df = df.loc[::-1].reset_index(drop = True) #reverse orders of results and reindex
df.head()

The winning numbers look like this (A	B C	D E  are balls that take range 0 to 100 each, Bonus and Extra numbers):

In [ ]:
df.head()

Next we need to normalize data.

In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

Lets define hyper params of or model

In [ ]:
number_of_rows= df.values.shape[0] #all our games
window_length = 5 #amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1] #balls count

Create train dataset and labels for each row. It should have format for keras lstm model (rows, window zise, balls)

In [ ]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

for i in range(0, number_of_rows-window_length):
    train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_features]

Shapes

In [ ]:
train.shape

In [ ]:
label.shape

In [ ]:
train[0]

In [ ]:
train[1]

In [ ]:
label[0]

In [ ]:
label[1]

## The LSTM model

In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense,Dropout

import numpy as np

batch_size = 25 

Training

In [ ]:
if os.path.exists('../input/israelilotto/'+filename+'.h5'):
    model = load_model('../input/israelilotto/'+filename+'.h5')
else:
    model = Sequential()
    model.add(LSTM(32,      
               input_shape=(window_length, number_of_features),
               return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32,           
               return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(number_of_features))
    model.compile(loss='mse', optimizer='rmsprop')
    model.fit(train, label,
          batch_size=64, epochs=10000)
    model.save('input/'+filename+'.h5')

## Prediction

Last step, we would like to predict the next results, the prediction will be based on the model and based on the last 5 results. And we will export everything to a csv file

In [ ]:
to_predict=df.iloc[-5:]
scaled_to_predict = scaler.transform(to_predict)

In [ ]:
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
data = scaler.inverse_transform(scaled_predicted_output_1).astype(int)
dfoutput = pd.DataFrame(data, columns=df.columns)
dfoutput.to_csv(''+filename+'.csv', index=False)  
dfoutput

Conclusion <br/>
We developed a LSTM model to forecast lotery game. Thanks for reading